# PREDICT "Item_Outlet_Sales" using Ml

Using XGBOOST Algorithm.
Data set:- Kaggle comp.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from xgboost import XGBRFRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,r2_score

In [2]:
df=pd.read_csv("E:\Train.csv")

In [3]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [4]:
## finding how many no. of data points and features
df.shape

(8523, 12)

In [5]:
# getting some info about data set
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


# checking for missing values

In [6]:
df.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [7]:
#mean value of item wait column
df["Item_Weight"].mean()

12.857645184135976

In [8]:
##filling the missing value in item-wt columns with mean value
df["Item_Weight"].fillna(df["Item_Weight"].mean(),inplace=True)

In [9]:
df.isnull().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

#Missing value in outlet-size (object type) with mode

In [10]:
df.Item_Weight = df.Item_Weight.fillna(df.groupby('Item_Type').Item_Weight.transform('mean'))

In [11]:
df.isnull().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

# data Analyse

In [12]:
df.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,8523.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.226124,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,9.310000,0.026989,93.826500,1987.000000,834.247400
50%,12.857645,0.053931,143.012800,1999.000000,1794.331000
75%,16.000000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


# Numeric feature

In [13]:
## plt the graph for all int
n=df.select_dtypes(include={"int64","float64"}).keys()

In [14]:
# var=[['Item_Weight', 'Item_Visibility', 'Item_MRP',
#        'Outlet_Establishment_Year', 'Item_Outlet_Sales']]
# for i ,var in enumerate(var):
#     plt.subplot(3,3,i+1)
#     sns.distplot(df[var],bins=20)

# Data preprocessing

Convert all text column to num

In [15]:
df.Item_Fat_Content.value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [16]:
df.Item_Fat_Content.replace(to_replace=['LF','low fat'], value='Low Fat', inplace=True)
df.Item_Fat_Content.replace(to_replace=['reg'], value='Regular', inplace=True)

In [17]:
one = ['Outlet_Size','Outlet_Location_Type','Outlet_Type']
df.Outlet_Size.value_counts(),df.Outlet_Location_Type.value_counts(),df.Outlet_Type.value_counts()

(Medium    2793
 Small     2388
 High       932
 Name: Outlet_Size, dtype: int64,
 Tier 3    3350
 Tier 2    2785
 Tier 1    2388
 Name: Outlet_Location_Type, dtype: int64,
 Supermarket Type1    5577
 Grocery Store        1083
 Supermarket Type3     935
 Supermarket Type2     928
 Name: Outlet_Type, dtype: int64)

# Lable encoding(convert str to int/float) one hot encoding

In [18]:
le = LabelEncoder()
enc = OneHotEncoder()

In [19]:
label = ['Item_Identifier','Item_Fat_Content','Item_Type','Outlet_Identifier','Outlet_Establishment_Year']
one = ['Outlet_Size','Outlet_Location_Type','Outlet_Type']
for col in label:
    df[col]=le.fit_transform(df[col]) 
for col in one:
    df[col]=le.fit_transform(df[col]) 

In [20]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,156,9.30,0,0.016047,4,249.8092,9,4,1,0,1,3735.1380
1,8,5.92,1,0.019278,14,48.2692,3,8,1,2,2,443.4228
2,662,17.50,0,0.016760,10,141.6180,9,4,1,0,1,2097.2700
3,1121,19.20,1,0.000000,6,182.0950,0,3,3,2,0,732.3800
4,1297,8.93,0,0.000000,9,53.8614,1,1,0,2,1,994.7052


# splitting

In [21]:
x=df.drop(["Item_Outlet_Sales"],axis=1)
y=df.Item_Outlet_Sales

In [22]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=2)

# Train ML model

In [23]:
reg=XGBRFRegressor()

In [24]:
reg.fit(x_train,y_train)

XGBRFRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bytree=1, gamma=0, gpu_id=-1, importance_type='gain',
               interaction_constraints='', max_delta_step=0, max_depth=6,
               min_child_weight=1, missing=nan, monotone_constraints='()',
               n_estimators=100, n_jobs=4, num_parallel_tree=100,
               objective='reg:squarederror', random_state=0, reg_alpha=0,
               scale_pos_weight=1, tree_method='exact', validate_parameters=1,
               verbosity=None)

In [25]:
training_pre=reg.predict(x_train)

In [26]:
# Rsquare value

In [27]:
r=r2_score(y_train,training_pre)
print("Rsquare prediction :", r)

Rsquare prediction : 0.6259958553737462
